# MCP LAMMPS Interactive Test Notebook

This notebook demonstrates the capabilities of MCP LAMMPS through multiple simulation examples. Each example focuses on setup and file generation, showing how to create LAMMPS-ready systems for different types of molecular simulations.

## What You'll Learn

1. **Single Organic Liquid**: Create a pure ethanol system
2. **Binary Mixture**: Set up ethanol-water mixtures
3. **Ternary Mixture**: Build complex solvent systems
4. **Custom Molecules**: Handle pharmaceutical compounds
5. **File Imports**: Work with MOL2, SDF, and PDB formats
6. **System Analysis**: Compare and validate generated systems

## Requirements

- Python 3.9+
- OpenFF Toolkit
- OpenFF Interchange
- RDKit
- mcp_lammps package

Let's get started!


## 1. Setup and Initialization

First, let's import the required modules and check that all dependencies are available.


In [1]:
import sys
import json
import logging
from pathlib import Path
from typing import Dict, Any, List
import warnings
warnings.filterwarnings('ignore')

# Add src to path if needed
sys.path.insert(0, str(Path.cwd().parent / "src"))

# Import mcp_lammps modules
from mcp_lammps.data_handler import DataHandler
from mcp_lammps.utils.openff_utils import openff_forcefield, OPENFF_AVAILABLE

# For data display
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Setup logging
logging.basicConfig(level=logging.WARNING)

print("✓ Imports successful!")
print(f"✓ Working directory: {Path.cwd()}")


✓ Imports successful!
✓ Working directory: /home/zhenghaowu/mcp_lammps/examples


In [2]:
# Check dependencies
print("Checking dependencies...\n")

dependencies_ok = True

if not OPENFF_AVAILABLE:
    print("✗ OpenFF Toolkit not available")
    print("  Install: pip install openff-toolkit openff-interchange")
    dependencies_ok = False
else:
    print("✓ OpenFF Toolkit available")
    try:
        from openff.toolkit import __version__ as openff_version
        print(f"  Version: {openff_version}")
    except:
        pass

try:
    from openff.interchange import __version__ as interchange_version
    print("✓ OpenFF Interchange available")
    print(f"  Version: {interchange_version}")
except ImportError:
    print("✗ OpenFF Interchange not available")
    dependencies_ok = False

try:
    import rdkit
    print("✓ RDKit available")
    print(f"  Version: {rdkit.__version__}")
except ImportError:
    print("✗ RDKit not available")
    dependencies_ok = False

if dependencies_ok:
    print("\n✓ All dependencies satisfied!")
else:
    print("\n✗ Missing dependencies - please install them first")


Checking dependencies...

✓ OpenFF Toolkit available
  Version: 0.18.0
✓ OpenFF Interchange available
  Version: 0.4.9
✓ RDKit available
  Version: 2023.09.5

✓ All dependencies satisfied!


In [3]:
# Setup workspace
output_dir = Path("notebook_output")
output_dir.mkdir(exist_ok=True)

# Initialize data handler
data_handler = DataHandler(output_dir)

# Dictionary to store results from all examples
results = {}

print(f"✓ Workspace created: {output_dir.absolute()}")
print(f"✓ Data handler initialized")
print(f"\nReady to run examples!")


2025-12-27 09:58:27,757 - mcp_lammps.data_handler - INFO - Data handler initialized with work directory: notebook_output


✓ Workspace created: /home/zhenghaowu/mcp_lammps/examples/notebook_output
✓ Data handler initialized

Ready to run examples!


## 2. Example 1: Single Organic Liquid (Ethanol)

**Goal**: Create the simplest case - a pure ethanol liquid system

**Key Concepts**:
- SMILES to 3D structure conversion
- OpenFF Sage 2.2.0 force field
- NAGL AM1-BCC charge assignment
- System validation

We'll create a liquid box with 100 ethanol molecules at the experimental density.


In [4]:
print("=" * 80)
print("EXAMPLE 1: Pure Ethanol Liquid")
print("=" * 80)

# Define system parameters
ethanol_params = {
    "smiles": "CCO",
    "molecule_count": 100,
    "target_density": 0.789,  # g/cm³ at 298 K
    "name": "ethanol_pure"
}

print(f"\nSystem Parameters:")
print(f"  SMILES: {ethanol_params['smiles']}")
print(f"  Number of molecules: {ethanol_params['molecule_count']}")
print(f"  Target density: {ethanol_params['target_density']} g/cm³")

# Create liquid box
print(f"\nCreating liquid box...")
try:
    molecules = [{
        "smiles": ethanol_params['smiles'],
        "count": ethanol_params['molecule_count'],
        "name": "ethanol"
    }]
    
    data_file = data_handler.create_liquid_box_file(
        molecules=molecules,
        target_density=ethanol_params['target_density'],
        box_type="cubic"
    )
    
    print(f"✓ Liquid box created successfully!")
    print(f"  Data file: {data_file.name}")
    
    # Load metadata
    metadata_file = data_file.with_suffix('.json')
    if metadata_file.exists():
        with open(metadata_file, 'r') as f:
            metadata = json.load(f)
        
        print(f"\n  System Information:")
        if 'system_info' in metadata:
            sys_info = metadata['system_info']
            print(f"    Total atoms: {sys_info.get('n_atoms', 'N/A')}")
            print(f"    Total bonds: {sys_info.get('n_bonds', 'N/A')}")
            print(f"    Total angles: {sys_info.get('n_angles', 'N/A')}")
            print(f"    Total dihedrals: {sys_info.get('n_dihedrals', 'N/A')}")
        
        if 'charge_information' in metadata:
            charge_info = metadata['charge_information']
            total_charge = charge_info.get('total_system_charge', 0)
            print(f"    Total charge: {total_charge:.6f} e")
            if abs(total_charge) < 1e-4:
                print(f"    ✓ System is electrically neutral")
        
        # Store results
        results['example1'] = {
            'name': 'Pure Ethanol',
            'data_file': str(data_file),
            'metadata': metadata,
            'params': ethanol_params
        }
    
    print(f"\n✓ Example 1 completed successfully!")
    
except Exception as e:
    print(f"✗ Error: {e}")
    import traceback
    traceback.print_exc()


2025-12-26 10:56:20,587 - mcp_lammps.utils.openff_utils - INFO - Created OpenFF molecule from SMILES: ethanol (CCO)


EXAMPLE 1: Pure Ethanol Liquid

System Parameters:
  SMILES: CCO
  Number of molecules: 100
  Target density: 0.789 g/cm³

Creating liquid box...


/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2025-12-26 10:56:20,903 - mcp_lammps.utils.openff_utils - INFO - Assigned am1bcc charges to molecule: ethanol
2025-12-26 10:56:20,906 - mcp_lammps.utils.openff_utils - INFO - Calculated box size: 21.32 Å for density 0.789 g/cm³
2025-12-26 10:56:20,907 - mcp_lammps.utils.openff_utils - INFO -   Total mass: 460

✓ Liquid box created successfully!
  Data file: liquid_box_1components.lmp

  System Information:

✓ Example 1 completed successfully!


## 3. Example 2: Binary Mixture (Ethanol + Water)

**Goal**: Create a multi-component system with two different molecules

**Key Concepts**:
- Multi-component mixtures
- Mole fraction composition
- Mixture density calculation
- Component interaction

We'll create a 50:50 mole ratio mixture of ethanol and water.


In [5]:
print("=" * 80)
print("EXAMPLE 2: Ethanol-Water Binary Mixture")
print("=" * 80)

# Define mixture parameters
binary_params = {
    "components": [
        {"smiles": "CCO", "count": 50, "name": "ethanol"},
        {"smiles": "O", "count": 50, "name": "water"}
    ],
    "target_density": 0.95,  # g/cm³ (approximate for 50:50 mixture)
    "name": "ethanol_water_mixture"
}

print(f"\nMixture Composition:")
for comp in binary_params['components']:
    print(f"  {comp['name']}: {comp['count']} molecules (SMILES: {comp['smiles']})")
print(f"  Target density: {binary_params['target_density']} g/cm³")
print(f"  Mole fraction: 50:50")

# Create mixture
print(f"\nCreating binary mixture...")
try:
    data_file = data_handler.create_liquid_box_file(
        molecules=binary_params['components'],
        target_density=binary_params['target_density'],
        box_type="cubic"
    )
    
    print(f"✓ Binary mixture created successfully!")
    print(f"  Data file: {data_file.name}")
    
    # Load metadata
    metadata_file = data_file.with_suffix('.json')
    if metadata_file.exists():
        with open(metadata_file, 'r') as f:
            metadata = json.load(f)
        
        print(f"\n  System Information:")
        if 'system_info' in metadata:
            sys_info = metadata['system_info']
            print(f"    Total molecules: {sys_info.get('n_molecules', 'N/A')}")
            print(f"    Total atoms: {sys_info.get('n_atoms', 'N/A')}")
            print(f"    Total bonds: {sys_info.get('n_bonds', 'N/A')}")
        
        if 'charge_information' in metadata:
            charge_info = metadata['charge_information']
            total_charge = charge_info.get('total_system_charge', 0)
            print(f"    Total charge: {total_charge:.6f} e")
            
            if 'molecule_charges' in charge_info:
                print(f"\n  Per-component charges:")
                for mol_charge in charge_info['molecule_charges']:
                    print(f"    {mol_charge.get('name', 'Unknown')}: {mol_charge.get('charge_per_molecule', 0):.6f} e/molecule")
        
        # Store results
        results['example2'] = {
            'name': 'Ethanol-Water Mixture',
            'data_file': str(data_file),
            'metadata': metadata,
            'params': binary_params
        }
    
    print(f"\n✓ Example 2 completed successfully!")
    
except Exception as e:
    print(f"✗ Error: {e}")
    import traceback
    traceback.print_exc()


2025-12-26 10:42:44,277 - mcp_lammps.utils.openff_utils - INFO - Created OpenFF molecule from SMILES: ethanol (CCO)


EXAMPLE 2: Ethanol-Water Binary Mixture

Mixture Composition:
  ethanol: 50 molecules (SMILES: CCO)
  water: 50 molecules (SMILES: O)
  Target density: 0.95 g/cm³
  Mole fraction: 50:50

Creating binary mixture...


/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2025-12-26 10:42:44,617 - mcp_lammps.utils.openff_utils - INFO - Assigned am1bcc charges to molecule: ethanol
2025-12-26 10:42:44,619 - mcp_lammps.utils.openff_utils - INFO - Detected water molecule, using TIP3P model
2025-12-26 10:42:44,626 - mcp_lammps.utils.openff_utils - INFO - Created TIP3P water molecul

✓ Binary mixture created successfully!
  Data file: liquid_box_2components.lmp

  System Information:

✓ Example 2 completed successfully!


## 4. Example 3: Ternary Mixture (Organic Solvents)

**Goal**: Create a complex three-component organic solvent system

**Key Concepts**:
- Multi-component systems (3+ components)
- Custom mole fractions
- Organic solvent properties
- Complex mixture behavior

We'll create a mixture of ethanol, acetone, and toluene with custom mole fractions (30:40:30).


In [6]:
print("=" * 80)
print("EXAMPLE 3: Ternary Organic Solvent Mixture")
print("=" * 80)

# Define ternary mixture parameters
ternary_params = {
    "components": [
        {"smiles": "CCO", "count": 30, "name": "ethanol"},
        {"smiles": "CC(=O)C", "count": 40, "name": "acetone"},
        {"smiles": "Cc1ccccc1", "count": 30, "name": "toluene"}
    ],
    "target_density": 0.85,  # g/cm³ (approximate for this mixture)
    "name": "ternary_solvent_mixture"
}

print(f"\nMixture Composition:")
total_mols = sum(c['count'] for c in ternary_params['components'])
for comp in ternary_params['components']:
    mole_frac = comp['count'] / total_mols * 100
    print(f"  {comp['name']}: {comp['count']} molecules ({mole_frac:.1f}%)")
    print(f"    SMILES: {comp['smiles']}")
print(f"  Target density: {ternary_params['target_density']} g/cm³")

# Create ternary mixture
print(f"\nCreating ternary mixture...")
try:
    data_file = data_handler.create_liquid_box_file(
        molecules=ternary_params['components'],
        target_density=ternary_params['target_density'],
        box_type="cubic"
    )
    
    print(f"✓ Ternary mixture created successfully!")
    print(f"  Data file: {data_file.name}")
    
    # Load metadata
    metadata_file = data_file.with_suffix('.json')
    if metadata_file.exists():
        with open(metadata_file, 'r') as f:
            metadata = json.load(f)
        
        print(f"\n  System Information:")
        if 'system_info' in metadata:
            sys_info = metadata['system_info']
            print(f"    Total molecules: {sys_info.get('n_molecules', 'N/A')}")
            print(f"    Total atoms: {sys_info.get('n_atoms', 'N/A')}")
            print(f"    Total bonds: {sys_info.get('n_bonds', 'N/A')}")
            print(f"    Total angles: {sys_info.get('n_angles', 'N/A')}")
            print(f"    Total dihedrals: {sys_info.get('n_dihedrals', 'N/A')}")
        
        if 'charge_information' in metadata:
            charge_info = metadata['charge_information']
            total_charge = charge_info.get('total_system_charge', 0)
            print(f"    Total charge: {total_charge:.6f} e")
            
            if 'molecule_charges' in charge_info:
                print(f"\n  Per-component charges:")
                for mol_charge in charge_info['molecule_charges']:
                    print(f"    {mol_charge.get('name', 'Unknown')}: {mol_charge.get('charge_per_molecule', 0):.6f} e/molecule")
        
        # Store results
        results['example3'] = {
            'name': 'Ternary Solvent Mixture',
            'data_file': str(data_file),
            'metadata': metadata,
            'params': ternary_params
        }
    
    print(f"\n✓ Example 3 completed successfully!")
    
except Exception as e:
    print(f"✗ Error: {e}")
    import traceback
    traceback.print_exc()


2025-12-26 10:46:21,195 - mcp_lammps.utils.openff_utils - INFO - Created OpenFF molecule from SMILES: ethanol (CCO)


EXAMPLE 3: Ternary Organic Solvent Mixture

Mixture Composition:
  ethanol: 30 molecules (30.0%)
    SMILES: CCO
  acetone: 40 molecules (40.0%)
    SMILES: CC(=O)C
  toluene: 30 molecules (30.0%)
    SMILES: Cc1ccccc1
  Target density: 0.85 g/cm³

Creating ternary mixture...


/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2025-12-26 10:46:21,531 - mcp_lammps.utils.openff_utils - INFO - Assigned am1bcc charges to molecule: ethanol
2025-12-26 10:46:21,571 - mcp_lammps.utils.openff_utils - INFO - Created OpenFF molecule from SMILES: acetone (CC(=O)C)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information

✓ Ternary mixture created successfully!
  Data file: liquid_box_3components.lmp

  System Information:

✓ Example 3 completed successfully!


## 5. Example 4: Custom Molecule (Ibuprofen)

**Goal**: Handle a complex pharmaceutical molecule

**Key Concepts**:
- Large organic molecules
- Multiple functional groups
- Pharmaceutical compounds
- Smaller system size for large molecules

We'll create a system with 20 ibuprofen molecules to demonstrate handling of complex structures.


In [5]:
print("=" * 80)
print("EXAMPLE 4: Ibuprofen (Complex Pharmaceutical Molecule)")
print("=" * 80)

# Define ibuprofen system parameters
ibuprofen_params = {
    "smiles": "CC(C)CC1=CC=C(C=C1)C(C)C(=O)O",  # Ibuprofen
    "molecule_count": 20,  # Smaller system due to molecular size
    "target_density": 1.03,  # g/cm³ (approximate for ibuprofen)
    "name": "ibuprofen_system"
}

print(f"\nMolecule Information:")
print(f"  Name: Ibuprofen (2-(4-isobutylphenyl)propionic acid)")
print(f"  SMILES: {ibuprofen_params['smiles']}")
print(f"  Number of molecules: {ibuprofen_params['molecule_count']}")
print(f"  Target density: {ibuprofen_params['target_density']} g/cm³")

# First, create a single molecule to show its properties
print(f"\nCreating single ibuprofen molecule for analysis...")
try:
    molecule = openff_forcefield.from_smiles(
        ibuprofen_params['smiles'],
        "ibuprofen_single",
        optimize=True
    )
    
    print(f"✓ Single molecule created")
    print(f"  Atoms: {molecule.n_atoms}")
    print(f"  Bonds: {molecule.n_bonds}")
    print(f"  Molecular formula: {molecule.hill_formula}")
    
    # Assign charges
    openff_forcefield.assign_charges(molecule)
    from openff.units import unit
    charges = [float(c.m_as(unit.elementary_charge)) for c in molecule.partial_charges]
    print(f"  Charge range: [{min(charges):.4f}, {max(charges):.4f}] e")
    
except Exception as e:
    print(f"  Note: Could not analyze single molecule: {e}")

# Create liquid box
print(f"\nCreating ibuprofen liquid system...")
try:
    molecules = [{
        "smiles": ibuprofen_params['smiles'],
        "count": ibuprofen_params['molecule_count'],
        "name": "ibuprofen"
    }]
    
    data_file = data_handler.create_liquid_box_file(
        molecules=molecules,
        target_density=ibuprofen_params['target_density'],
        box_type="cubic"
    )
    
    print(f"✓ Ibuprofen system created successfully!")
    print(f"  Data file: {data_file.name}")
    
    # Load metadata
    metadata_file = data_file.with_suffix('.json')
    if metadata_file.exists():
        with open(metadata_file, 'r') as f:
            metadata = json.load(f)
        
        print(f"\n  System Information:")
        if 'system_info' in metadata:
            sys_info = metadata['system_info']
            print(f"    Total atoms: {sys_info.get('n_atoms', 'N/A')}")
            print(f"    Total bonds: {sys_info.get('n_bonds', 'N/A')}")
            print(f"    Total angles: {sys_info.get('n_angles', 'N/A')}")
            print(f"    Total dihedrals: {sys_info.get('n_dihedrals', 'N/A')}")
            print(f"    Atoms per molecule: {sys_info.get('n_atoms', 0) // ibuprofen_params['molecule_count']}")
        
        if 'charge_information' in metadata:
            charge_info = metadata['charge_information']
            total_charge = charge_info.get('total_system_charge', 0)
            print(f"    Total charge: {total_charge:.6f} e")
        
        # Store results
        results['example4'] = {
            'name': 'Ibuprofen System',
            'data_file': str(data_file),
            'metadata': metadata,
            'params': ibuprofen_params
        }
    
    print(f"\n✓ Example 4 completed successfully!")
    
except Exception as e:
    print(f"✗ Error: {e}")
    import traceback
    traceback.print_exc()


2025-12-26 10:56:22,206 - mcp_lammps.utils.openff_utils - INFO - Created OpenFF molecule from SMILES: ibuprofen_single (CC(C)CC1=CC=C(C=C1)C(C)C(=O)O)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


EXAMPLE 4: Ibuprofen (Complex Pharmaceutical Molecule)

Molecule Information:
  Name: Ibuprofen (2-(4-isobutylphenyl)propionic acid)
  SMILES: CC(C)CC1=CC=C(C=C1)C(C)C(=O)O
  Number of molecules: 20
  Target density: 1.03 g/cm³

Creating single ibuprofen molecule for analysis...
✓ Single molecule created
  Atoms: 33
  Bonds: 33
  Molecular formula: C13H18O2


/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2025-12-26 10:56:53,429 - mcp_lammps.utils.openff_utils - INFO - Assigned am1bcc charges to molecule: ibuprofen_single
2025-12-26 10:56:53,529 - mcp_lammps.utils.openff_utils - INFO - Created OpenFF molecule from SMILES: ibuprofen (CC(C)CC1=CC=C(C=C1)C(C)C(=O)O)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


  Charge range: [-0.6081, 0.6391] e

Creating ibuprofen liquid system...


/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2025-12-26 10:57:15,589 - mcp_lammps.utils.openff_utils - INFO - Assigned am1bcc charges to molecule: ibuprofen
2025-12-26 10:57:15,591 - mcp_lammps.utils.openff_utils - INFO - Calculated box size: 18.81 Å for density 1.03 g/cm³
2025-12-26 10:57:15,593 - mcp_lammps.utils.openff_utils - INFO -   Total mass: 4125.64 amu, Volume: 6651.41 ų
2025-12-26 10:57:15,593 - mcp_lammps.utils.openff_utils - INFO - Using Packmol to pack

✓ Ibuprofen system created successfully!
  Data file: liquid_box_1components.lmp

  System Information:

✓ Example 4 completed successfully!


## 6. Example 5: File Format Import Demonstration

**Goal**: Demonstrate importing molecules from various file formats

**Key Concepts**:
- File format interoperability
- MOL2, SDF, PDB support
- Format auto-detection
- Converting external structures to LAMMPS

For this example, we'll create sample molecular files and import them to show the capability.


In [4]:
print("=" * 80)
print("EXAMPLE 5: File Format Import Demonstration")
print("=" * 80)

print("\nThis example demonstrates the file import capabilities.")
print("We'll create molecules from SMILES and show how they could be")
print("imported from various file formats (MOL2, SDF, PDB).")

# For demonstration, we'll use the import_smiles_structure method
# which is the underlying mechanism for file imports
print("\n--- Importing Methanol from SMILES ---")
#try:

methanol_file = data_handler.import_smiles_structure(
    smiles="CO",
    molecule_name="methanol_demo",
    optimize_geometry=True
)

print(f"✓ Methanol structure imported")
print(f"  File: {methanol_file.name}")
print(f"  Format: LAMMPS data file")

# Check for metadata
metadata_file = methanol_file.with_suffix('.json')
if metadata_file.exists():
    with open(metadata_file, 'r') as f:
        metadata = json.load(f)
    print(f"  Force field: {metadata.get('force_field', 'N/A')}")
    print(f"  Charge method: {metadata.get('charge_method', 'N/A')}")
    
    if 'system_info' in metadata:
        sys_info = metadata['system_info']
        print(f"  Atoms: {sys_info.get('n_atoms', 'N/A')}")
        print(f"  Bonds: {sys_info.get('n_bonds', 'N/A')}")

results['example5_methanol'] = {
    'name': 'Methanol (from SMILES)',
    'data_file': str(methanol_file),
    'source_format': 'SMILES'
}
    
#except Exception as e:
#    print(f"✗ Error importing methanol: {e}")

print("\n--- Importing Benzene from SMILES ---")
try:
    benzene_file = data_handler.import_smiles_structure(
        smiles="c1ccccc1",
        molecule_name="benzene_demo",
        optimize_geometry=True
    )
    
    print(f"✓ Benzene structure imported")
    print(f"  File: {benzene_file.name}")
    
    # Check for metadata
    metadata_file = benzene_file.with_suffix('.json')
    if metadata_file.exists():
        with open(metadata_file, 'r') as f:
            metadata = json.load(f)
        
        if 'system_info' in metadata:
            sys_info = metadata['system_info']
            print(f"  Atoms: {sys_info.get('n_atoms', 'N/A')}")
            print(f"  Bonds: {sys_info.get('n_bonds', 'N/A')}")
    
    results['example5_benzene'] = {
        'name': 'Benzene (from SMILES)',
        'data_file': str(benzene_file),
        'source_format': 'SMILES'
    }
    
except Exception as e:
    print(f"✗ Error importing benzene: {e}")

print("\n--- File Format Support Summary ---")
print("MCP LAMMPS supports importing from:")
print("  ✓ SMILES strings (demonstrated above)")
print("  ✓ MOL2 files (via import_molecular_file tool)")
print("  ✓ SDF files (via import_molecular_file tool)")
print("  ✓ PDB files (via import_molecular_file tool)")
print("  ✓ Auto-detection of file formats")
print("\nAll formats are converted to LAMMPS data files with OpenFF force field parameters.")

print(f"\n✓ Example 5 completed successfully!")


2025-12-27 09:58:39,073 - mcp_lammps.utils.openff_utils - INFO - Created OpenFF molecule from SMILES: methanol_demo (CO)


EXAMPLE 5: File Format Import Demonstration

This example demonstrates the file import capabilities.
We'll create molecules from SMILES and show how they could be
imported from various file formats (MOL2, SDF, PDB).

--- Importing Methanol from SMILES ---


/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2025-12-27 09:58:39,329 - mcp_lammps.utils.openff_utils - INFO - Assigned am1bcc charges to molecule: methanol_demo
2025-12-27 09:58:39,330 - mcp_lammps.utils.openff_utils - INFO - Created topology with 1 molecules, 6 atoms
2025-12-27 09:58:39,389 - openff.interchange.smirnoff._nonbonded - INFO - Preset charg

✓ Methanol structure imported
  File: methanol_demo_from_smiles.lmp
  Format: LAMMPS data file
  Force field: openff-sage-2.2.0
  Charge method: am1bcc

--- Importing Benzene from SMILES ---


/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/zhenghaowu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2025-12-27 09:58:39,809 - mcp_lammps.utils.openff_utils - INFO - Assigned am1bcc charges to molecule: benzene_demo
2025-12-27 09:58:39,812 - mcp_lammps.utils.openff_utils - INFO - Created topology with 1 molecules, 12 atoms
2025-12-27 09:58:39,888 - openff.interchange.smirnoff._nonbonded - INFO - Preset charges applied to atom index 0
2025-12-27 09:58:39,889 - openff.interchange.smirnoff._nonbonded - INFO - Preset charges applied to atom index 1
2025-12-27 09:58:39,890 - openff.interchange.smirnoff._nonbonded - INFO - Preset charges a

✓ Benzene structure imported
  File: benzene_demo_from_smiles.lmp

--- File Format Support Summary ---
MCP LAMMPS supports importing from:
  ✓ SMILES strings (demonstrated above)
  ✓ MOL2 files (via import_molecular_file tool)
  ✓ SDF files (via import_molecular_file tool)
  ✓ PDB files (via import_molecular_file tool)
  ✓ Auto-detection of file formats

All formats are converted to LAMMPS data files with OpenFF force field parameters.

✓ Example 5 completed successfully!


## 7. System Analysis and Comparison

Now let's compare all the systems we've created and analyze their properties.


In [5]:
print("=" * 80)
print("SYSTEM COMPARISON AND ANALYSIS")
print("=" * 80)

# Collect data from all examples
comparison_data = []

for example_id, example_data in results.items():
    if 'metadata' in example_data and 'system_info' in example_data['metadata']:
        sys_info = example_data['metadata']['system_info']
        charge_info = example_data['metadata'].get('charge_information', {})
        
        # Get file size
        data_file = Path(example_data['data_file'])
        file_size_kb = data_file.stat().st_size / 1024 if data_file.exists() else 0
        
        comparison_data.append({
            'Example': example_data['name'],
            'Molecules': sys_info.get('n_molecules', 'N/A'),
            'Atoms': sys_info.get('n_atoms', 'N/A'),
            'Bonds': sys_info.get('n_bonds', 'N/A'),
            'Angles': sys_info.get('n_angles', 'N/A'),
            'Dihedrals': sys_info.get('n_dihedrals', 'N/A'),
            'Total Charge': f"{charge_info.get('total_system_charge', 0):.6f}",
            'File Size (KB)': f"{file_size_kb:.1f}"
        })

# Create DataFrame for nice display
if comparison_data:
    df = pd.DataFrame(comparison_data)
    print("\n### System Statistics Comparison ###\n")
    print(df.to_string(index=False))
    
    print("\n\n### Summary Statistics ###")
    print(f"Total systems created: {len(comparison_data)}")
    
    # Calculate totals
    total_atoms = sum([int(d['Atoms']) for d in comparison_data if d['Atoms'] != 'N/A'])
    total_bonds = sum([int(d['Bonds']) for d in comparison_data if d['Bonds'] != 'N/A'])
    total_size = sum([float(d['File Size (KB)']) for d in comparison_data])
    
    print(f"Total atoms across all systems: {total_atoms:,}")
    print(f"Total bonds across all systems: {total_bonds:,}")
    print(f"Total file size: {total_size:.1f} KB ({total_size/1024:.2f} MB)")
else:
    print("\nNo comparison data available")

print("\n" + "=" * 80)


SYSTEM COMPARISON AND ANALYSIS

No comparison data available



In [6]:
# Display file locations
print("=" * 80)
print("GENERATED FILES")
print("=" * 80)

print(f"\nAll files are located in: {output_dir.absolute()}\n")

for example_id, example_data in results.items():
    print(f"### {example_data['name']} ###")
    data_file = Path(example_data['data_file'])
    
    if data_file.exists():
        print(f"  Data file: {data_file.name}")
        
        # Check for associated files
        script_file = data_file.with_suffix('.in')
        if script_file.exists():
            print(f"  Input script: {script_file.name}")
        
        metadata_file = data_file.with_suffix('.json')
        if metadata_file.exists():
            print(f"  Metadata: {metadata_file.name}")
    
    print()

print("=" * 80)


GENERATED FILES

All files are located in: /home/zhenghaowu/mcp_lammps/examples/notebook_output

### Methanol (from SMILES) ###
  Data file: methanol_demo_from_smiles.lmp
  Metadata: methanol_demo_from_smiles.json

### Benzene (from SMILES) ###
  Data file: benzene_demo_from_smiles.lmp
  Metadata: benzene_demo_from_smiles.json



## 8. Next Steps and Resources

Congratulations! You've successfully created multiple LAMMPS-ready molecular systems using MCP LAMMPS.


### How to Run Simulations with LAMMPS

To run a simulation with the generated files:

```bash
# Navigate to the output directory
cd notebook_output/input

# Run LAMMPS with the input script
lmp -in <filename>.in

# Or if you have MPI:
mpirun -np 4 lmp -in <filename>.in
```

### Analysis Tools Available in MCP LAMMPS

Once you've run simulations, you can use these MCP tools for analysis:

1. **Trajectory Analysis**
   - `analyze_trajectory`: Analyze MD trajectories
   - `calculate_rdf`: Radial distribution functions
   
2. **Property Calculations**
   - `calculate_liquid_density`: Density with error analysis
   - `calculate_transport_properties`: Viscosity, diffusion, thermal conductivity
   - `calculate_thermodynamic_properties`: Heat capacity, compressibility
   
3. **Monitoring**
   - `get_simulation_status`: Check simulation progress
   - `get_simulation_logs`: View simulation logs
   - `monitor_simulation`: Real-time monitoring

### Key Features Demonstrated

✓ **SMILES to LAMMPS**: Convert molecular structures from SMILES strings  
✓ **OpenFF Force Field**: Automatic parameterization with Sage 2.2.0  
✓ **NAGL Charges**: Neural network-based AM1-BCC charges  
✓ **Multi-component Systems**: Binary and ternary mixtures  
✓ **Complex Molecules**: Pharmaceutical compounds like ibuprofen  
✓ **File Interoperability**: Support for multiple file formats  

### Documentation and Resources

- **MCP LAMMPS Documentation**: `../docs/`
- **API Reference**: `../docs/api-reference.md`
- **User Guide**: `../docs/user-guide.md`
- **Examples**: `../docs/examples.md`
- **Migration Guide**: `../docs/MIGRATION_GUIDE.md`

### OpenFF Resources

- [OpenFF Toolkit Documentation](https://docs.openforcefield.org/)
- [OpenFF Sage Force Field](https://openforcefield.org/community/news/general/sage2.2.0-release/)
- [NAGL Charges](https://docs.openforcefield.org/projects/toolkit/en/stable/users/charge_models.html)

### LAMMPS Resources

- [LAMMPS Documentation](https://docs.lammps.org/)
- [LAMMPS Tutorials](https://lammpstutorials.github.io/)
- [LAMMPS Forum](https://matsci.org/c/lammps/)

### Troubleshooting

**Issue**: OpenFF dependencies not found  
**Solution**: `pip install openff-toolkit openff-interchange openff-units`

**Issue**: NAGL charge assignment fails  
**Solution**: `pip install openff-nagl` or use alternative charge method

**Issue**: Packmol not found (for liquid boxes)  
**Solution**: `sudo apt install packmol` (Ubuntu) or `brew install packmol` (macOS)

**Issue**: LAMMPS not found  
**Solution**: Install LAMMPS with Python interface: [Installation Guide](https://docs.lammps.org/Install.html)

### What's Next?

1. **Run Simulations**: Use LAMMPS to run MD simulations with the generated files
2. **Analyze Results**: Use MCP LAMMPS analysis tools on trajectory data
3. **Customize Systems**: Modify parameters to create your own systems
4. **Explore Force Fields**: Try different force field options
5. **Scale Up**: Create larger systems for production simulations

### Contact and Support

- **GitHub Issues**: Report bugs or request features
- **Documentation**: Check the docs folder for detailed guides
- **Examples**: See the examples folder for more use cases


In [ ]:
print("=" * 80)
print("NOTEBOOK COMPLETED SUCCESSFULLY!")
print("=" * 80)

print("\n✓ All examples completed")
print(f"✓ {len(results)} systems created")
print(f"✓ Files saved to: {output_dir.absolute()}")

print("\n" + "=" * 80)
print("Thank you for using MCP LAMMPS!")
print("=" * 80)
